我这里分析的是所有形态的数据，并且都是区分大于0和小于0的。

In [1]:
import pandas as pd
import numpy as np
import matplotlib
# matplotlib.use('agg')
import matplotlib.pyplot as plt
%matplotlib inline
import os
from tqdm import tqdm
import talib

In [2]:
# 这里导入数据
df = pd.read_csv("../数据获取/聚宽数据/最新数据.csv")
df.columns = ['id','i', 'Date','Open','High','Low','Close','Volume']
df['id2'] =df['id'].shift()
df['i'] = range(len(df))
ids = list(df.loc[df['id'] != df['id2'], 'i']) # 这个是单个股票序号的起始。
ids.append(len(df)-1) # 添加最后1天。
df.head()

,id,i,Date,Open,High,Low,Close,Volume,id2
0,000001.XSHE,0,2018-12-24,8.91,8.96,8.83,8.93,53694517.0,NaN
1,000001.XSHE,1,2018-12-25,8.81,8.94,8.73,8.86,61867885.0,000001.XSHE
2,000001.XSHE,2,2018-12-26,8.87,8.93,8.79,8.82,44415942.0,000001.XSHE
3,000001.XSHE,3,2018-12-27,8.96,9.00,8.80,8.80,65873247.0,000001.XSHE
4,000001.XSHE,4,2018-12-28,8.83,8.97,8.83,8.89,60812019.0,000001.XSHE


In [3]:
xingtai = {
    "CDL2CROWS":talib.CDL2CROWS,
    "CDL3BLACKCROWS":talib.CDL3BLACKCROWS,
    "CDL3INSIDE":talib.CDL3INSIDE,
    "CDL3LINESTRIKE":talib.CDL3LINESTRIKE,
    "CDL3OUTSIDE":talib.CDL3OUTSIDE,
    "CDL3STARSINSOUTH":talib.CDL3STARSINSOUTH,
    "CDL3WHITESOLDIERS":talib.CDL3WHITESOLDIERS,
    "CDLABANDONEDBABY":talib.CDLABANDONEDBABY,
    "CDLADVANCEBLOCK":talib.CDLADVANCEBLOCK,
    "CDLBELTHOLD":talib.CDLBELTHOLD,
    "CDLBREAKAWAY":talib.CDLBREAKAWAY,
    "CDLCLOSINGMARUBOZU":talib.CDLCLOSINGMARUBOZU,
    "CDLCONCEALBABYSWALL":talib.CDLCONCEALBABYSWALL,
    "CDLCOUNTERATTACK":talib.CDLCOUNTERATTACK,
    "CDLDARKCLOUDCOVER":talib.CDLDARKCLOUDCOVER,
    "CDLDOJI":talib.CDLDOJI,
    "CDLDOJISTAR":talib.CDLDOJISTAR,
    "CDLDRAGONFLYDOJI":talib.CDLDRAGONFLYDOJI,
    "CDLENGULFING":talib.CDLENGULFING,
    "CDLEVENINGDOJISTAR":talib.CDLEVENINGDOJISTAR,
    "CDLEVENINGSTAR":talib.CDLEVENINGSTAR,
    "CDLGAPSIDESIDEWHITE":talib.CDLGAPSIDESIDEWHITE,
    "CDLGRAVESTONEDOJI":talib.CDLGRAVESTONEDOJI,
    "CDLHAMMER":talib.CDLHAMMER,
    "CDLHANGINGMAN":talib.CDLHANGINGMAN,
    "CDLHARAMI":talib.CDLHARAMI,
    "CDLHARAMICROSS":talib.CDLHARAMICROSS,
    "CDLHIGHWAVE":talib.CDLHIGHWAVE,
    "CDLHIKKAKE":talib.CDLHIKKAKE,
    "CDLHIKKAKEMOD":talib.CDLHIKKAKEMOD,
    "CDLHOMINGPIGEON":talib.CDLHOMINGPIGEON,
    "CDLIDENTICAL3CROWS":talib.CDLIDENTICAL3CROWS,
    "CDLINNECK":talib.CDLINNECK,
    "CDLINVERTEDHAMMER":talib.CDLINVERTEDHAMMER,
    "CDLKICKING":talib.CDLKICKING,
    "CDLKICKINGBYLENGTH":talib.CDLKICKINGBYLENGTH,
    "CDLLADDERBOTTOM":talib.CDLLADDERBOTTOM,
    "CDLLONGLEGGEDDOJI":talib.CDLLONGLEGGEDDOJI,
    "CDLLONGLINE":talib.CDLLONGLINE,
    "CDLMARUBOZU":talib.CDLMARUBOZU,
    "CDLMATCHINGLOW":talib.CDLMATCHINGLOW,
    "CDLMATHOLD":talib.CDLMATHOLD,
    "CDLMORNINGDOJISTAR":talib.CDLMORNINGDOJISTAR,
    "CDLMORNINGSTAR":talib.CDLMORNINGSTAR,
    "CDLONNECK":talib.CDLONNECK,
    "CDLPIERCING":talib.CDLPIERCING,
    "CDLRICKSHAWMAN":talib.CDLRICKSHAWMAN,
    "CDLRISEFALL3METHODS":talib.CDLRISEFALL3METHODS,
    "CDLSEPARATINGLINES":talib.CDLSEPARATINGLINES,
    "CDLSHOOTINGSTAR":talib.CDLSHOOTINGSTAR,
    "CDLSHORTLINE":talib.CDLSHORTLINE,
    "CDLSPINNINGTOP":talib.CDLSPINNINGTOP,
    "CDLSTALLEDPATTERN":talib.CDLSTALLEDPATTERN ,
    "CDLSTICKSANDWICH":talib.CDLSTICKSANDWICH,
    "CDLTAKURI":talib.CDLTAKURI,
    "CDLTASUKIGAP":talib.CDLTASUKIGAP,
    "CDLTHRUSTING":talib.CDLTHRUSTING,
    "CDLTRISTAR":talib.CDLTRISTAR,
    "CDLUNIQUE3RIVER":talib.CDLUNIQUE3RIVER,
    "CDLUPSIDEGAP2CROWS":talib.CDLUPSIDEGAP2CROWS,
    "CDLXSIDEGAP3METHODS":talib.CDLXSIDEGAP3METHODS,
    
}

In [4]:
after_days = 5 # 未来5天的
after_up = {key:[[] for _ in range(after_days)] for key in xingtai}  # 几天关于上涨的
after_down = {key:[[] for _ in range(after_days)] for key in xingtai} # 未来几天关于下跌的。
with tqdm(total=len(set(df['id']))) as pbar:
    for j in range(len(ids)-1):
        # 对每一个股票进行遍历
        df2 = df.iloc[ids[j]:ids[j+1],:].copy() # 这里筛选一只股票。
        for i in range(after_days):
            df2['Close{}'.format(i+1)] = df2['Close'].shift(0-(i+1)) # 未来的收盘价移动
        # 然后这里判断形态
        for key in xingtai:
            df2[key] = xingtai[key](df2['Open'], df2['High'], df2['Low'], df2['Close'])
        df2.dropna(axis=0, how='any', inplace=True) # 我这里不需要这个空值，直接删除吧。
        for i in range(after_days):
            # 统计这几天的情况
            for key in xingtai:
                # 这里还要判断形态
                df3 = df2.loc[df2[key] > 0, :].copy() # 先筛选
                after_up[key][i].extend(
                    ((df3['Close{}'.format(i+1)]-df3['Close'])/df3['Close']*100).tolist()
                )
                df4 = df2.loc[df2[key] < 0, :].copy() # 先筛选
                after_down[key][i].extend(
                    ((df3['Close{}'.format(i+1)]-df3['Close'])/df3['Close']*100).tolist()
                )

        pbar.update(1)

100%|████████████████████████████████████████████████████████████████████████████| 4275/4275 [1:18:44<00:00,  1.11s/it]


In [9]:
print("***********************************上涨***************************************************")
for key in xingtai:
    print("{} - {} 次".format(key, len(after_up[key][0])))
    if len(after_up[key][0]) > 0:
        for i in range(after_days):
            print("{}:往后{}, \t平均涨幅:{:.2f}, \t胜率:{:.2f}%, \t差不多涨停的概率:{:.2f}%".format(
                key,
                i+1,
                sum(after_up[key][i])/len(after_up[key][i]),
                len([i for i in after_up[key][i] if i > 0])/len(after_up[key][i])*100,
                len([i for i in after_up[key][i] if i > 9])/len(after_up[key][i])*100
            ))
    print("*"*120)

***********************************上涨***************************************************
CDL2CROWS - 0 次
************************************************************************************************************************
CDL3BLACKCROWS - 0 次
************************************************************************************************************************
CDL3INSIDE - 21073 次
CDL3INSIDE:往后1, 	平均涨幅:0.30, 	胜率:48.95%, 	差不多涨停的概率:2.65%
CDL3INSIDE:往后2, 	平均涨幅:0.52, 	胜率:51.87%, 	差不多涨停的概率:4.09%
CDL3INSIDE:往后3, 	平均涨幅:0.64, 	胜率:51.31%, 	差不多涨停的概率:6.13%
CDL3INSIDE:往后4, 	平均涨幅:0.65, 	胜率:50.65%, 	差不多涨停的概率:7.70%
CDL3INSIDE:往后5, 	平均涨幅:0.49, 	胜率:48.28%, 	差不多涨停的概率:8.85%
************************************************************************************************************************
CDL3LINESTRIKE - 2154 次
CDL3LINESTRIKE:往后1, 	平均涨幅:0.20, 	胜率:53.34%, 	差不多涨停的概率:1.53%
CDL3LINESTRIKE:往后2, 	平均涨幅:0.47, 	胜率:54.55%, 	差不多涨停的概率:3.30%
CDL3LINESTRIKE:往后3, 	平均涨幅:0.39, 	胜率:53.95%, 	差不多涨停的概率:4.83%
CDL3LI

In [8]:
print("***********************************下跌***************************************************")
for key in xingtai:
    print("{} - {} 次".format(key, len(after_up[key][0])))
    if len(after_down[key][0]) > 0:
        for i in range(after_days):
            print("{}:往后{}, \t平均涨幅:{:.2f}, \t胜率:{:.2f}%, \t差不多跌停的概率:{:.2f}%".format(
                key,
                i+1,
                sum(after_down[key][i])/len(after_down[key][i]),
                len([i for i in after_down[key][i] if i < 0])/len(after_down[key][i])*100,
                len([i for i in after_down[key][i] if i < -9])/len(after_down[key][i])*100
            ))
    print("*"*120)

***********************************下跌***************************************************
CDL2CROWS - 0 次
************************************************************************************************************************
CDL3BLACKCROWS - 0 次
************************************************************************************************************************
CDL3INSIDE - 21073 次
CDL3INSIDE:往后1, 	平均涨幅:0.30, 	胜率:48.55%, 	差不多跌停的概率:0.40%
CDL3INSIDE:往后2, 	平均涨幅:0.52, 	胜率:46.47%, 	差不多跌停的概率:1.65%
CDL3INSIDE:往后3, 	平均涨幅:0.64, 	胜率:47.52%, 	差不多跌停的概率:2.90%
CDL3INSIDE:往后4, 	平均涨幅:0.65, 	胜率:48.28%, 	差不多跌停的概率:4.18%
CDL3INSIDE:往后5, 	平均涨幅:0.49, 	胜率:50.70%, 	差不多跌停的概率:5.78%
************************************************************************************************************************
CDL3LINESTRIKE - 2154 次
CDL3LINESTRIKE:往后1, 	平均涨幅:0.20, 	胜率:44.34%, 	差不多跌停的概率:0.79%
CDL3LINESTRIKE:往后2, 	平均涨幅:0.47, 	胜率:43.87%, 	差不多跌停的概率:2.27%
CDL3LINESTRIKE:往后3, 	平均涨幅:0.39, 	胜率:44.89%, 	差不多跌停的概率:3.44%
CDL3LI

In [ ]:
# 上边我看了形态分析，但好像概率都是对半分的